In [1]:
from pynubank import Nubank, MockHttpClient
import json
import pandas as pd
from datetime import datetime
from delta import get_metrics
from gsheet import insert_values_in_gsheets
import os

current_week = datetime.now().isocalendar().week
current_datetime = datetime.now().strftime('%Y-%m-%d')
nu = Nubank()

with open("credentials.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

#if os.path.exists('fresh_token.txt') == False:
fresh_token = nu.authenticate_with_cert(data["login"], data["pwd"], "cert.p12")
with open("fresh_token.txt", 'w') as token_file:
    token_file.write(fresh_token)
print(fresh_token)

Exception ignored from cffi callback <function _verify_callback at 0x00000206F2D1F940>:
Traceback (most recent call last):
  File "C:\Users\caio.lauro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\OpenSSL\SSL.py", line 305, in wrapper
    @wraps(callback)
KeyboardInterrupt: 


SSLError: HTTPSConnectionPool(host='prod-global-auth.nubank.com.br', port=443): Max retries exceeded with url: /api/token (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [9]:
pd.read_csv(r"C:\Users\caio.lauro\Documents\3_my_projects\nu_scrap\2022_01_17__12_50_04_raw_cc_trans.csv")

,Unnamed: 0,description,category,amount,time,source,title,amount_without_iof,account,id,...,details.charges.count,details.charges.amount,details.lat,details.lon,details.fx.currency_origin,details.fx.amount_origin,details.fx.amount_usd,details.fx.precise_amount_origin,details.fx.precise_amount_usd,details.fx.exchange_rate
0,0,Ifood *Ifood,transaction,2487,2022-01-16T15:36:41Z,upfront_national,supermercado,2487.0,59ea3883-de78-4530-af24-91c5eb9a0bed,61e43b89-1d3c-47a6-ab08-e4d43b4dec98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Netflix.Com,transaction,3990,2022-01-16T13:08:49Z,upfront_national,serviços,3990.0,59ea3883-de78-4530-af24-91c5eb9a0bed,61e418e2-67c5-4c05-9b27-7fc8cb7d60ee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Rappi*Rappi Brasil Int,transaction,6192,2022-01-12T15:13:48Z,upfront_national,transporte,6192.0,59ea3883-de78-4530-af24-91c5eb9a0bed,61def02d-b283-4ca6-bc77-49740a612a19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Amazonprimebr,transaction,990,2022-01-10T00:18:43Z,upfront_national,educação,990.0,59ea3883-de78-4530-af24-91c5eb9a0bed,61db7b64-030e-483a-bd07-274c331c35fc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Bg Norte Alimentos,transaction,5200,2022-01-09T22:51:46Z,upfront_national,restaurante,5200.0,59ea3883-de78-4530-af24-91c5eb9a0bed,61db6707-14ba-4d5e-b9bb-c75f0b64d402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,861,Paypal do Brasil,transaction,6278,2017-11-08T15:15:06Z,NaN,outros,NaN,NaN,5a031f7a-37ed-4504-b0c2-8d60e82ef50b,...,4.0,1569.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
862,862,Esporte Clube Sirio,transaction,1250,2017-11-07T20:43:00Z,NaN,lazer,NaN,NaN,5a021ad6-ee9d-4b6c-96f6-3b1ab455a493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863,863,Pag*Cavc,transaction,200,2017-11-06T19:26:44Z,NaN,outros,NaN,NaN,5a00b775-19c2-4657-8071-5f57966761c0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
864,864,Pao de Acucar Lj,transaction,803,2017-11-05T22:27:42Z,NaN,supermercado,NaN,NaN,59ff905e-c6e8-4d7d-95bb-6f7952c22542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_nc = pd.read_csv(r"C:\Users\caio.lauro\Documents\3_my_projects\nu_scrap\2022_01_17__12_50_04_raw_nc_trans.csv")
df_nc['amount'] = df_nc['amount'].apply(lambda x: float(x))
df_nc.where(df_nc['amount']== 3363.19)['__typename'].dropna()
nd = df_nc.where(df_nc['__typename']=='BillPaymentEvent')
nd.columns
nd.loc[:,['__typename','title','detail','amount']].where(nd['title'] == 'Pagamento da fatura').dropna()

,__typename,title,detail,amount
8,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 3.363,19",3363.19
17,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 890,00",890.00
28,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 3.275,03",3275.03
59,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 2.817,17",2817.17
79,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 1.465,12",1465.12
...,...,...,...,...
491,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 653,25",653.25
496,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 70,00",70.00
497,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 311,63",311.63
504,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 342,07",342.07


In [62]:
df_nc = pd.read_csv(r"C:\Users\caio.lauro\Documents\3_my_projects\nu_scrap\2022_01_17__12_50_04_raw_nc_trans.csv")
df_nc

,Unnamed: 0,id,__typename,title,detail,postDate,amount,originAccount,destinationAccount
0,0,61e2e7f4-3cc3-459f-a315-b3365dab04bb,TransferInEvent,Transferência recebida,"R$ 5.004,42",2022-01-15,5004.42,{'name': 'Caio Saber Lauro'},NaN
1,1,61e14dcc-17cf-4e0a-9aeb-ff8724f416a6,TransferInEvent,Transferência recebida,"R$ 1.825,20",2022-01-14,1825.20,{'name': 'Caio Saber Lauro'},NaN
2,2,61df6663-800c-43c0-a73e-f4cf014e4023,TransferInEvent,Transferência recebida,"R$ 580,00",2022-01-12,580.00,{'name': 'Elis Valeria Anginski Cotosky'},NaN
3,3,61dd6cd9-4e99-42f2-9681-2deb0efbeede,TransferInEvent,Transferência recebida,"R$ 5.121,00",2022-01-11,5121.00,{'name': 'Caio Saber Lauro'},NaN
4,4,61dd6a5c-0e17-4a00-80d8-6953f0fa7d61,PixTransferOutEvent,Transferência enviada,"Adyen do Brasil LTDA\nR$ 4.999,00",2022-01-11,4999.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...
510,510,5aa2219d-84b2-4abb-b8b5-fff6d7cd19e2,TransferInEvent,Depósito recebido,"R$ 850,00",2018-03-08,850.00,NaN,NaN
511,511,5a9af0a4-e9ef-4f0a-bd14-e98b83da6bf8,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 171,44",2018-03-03,171.44,NaN,NaN
512,512,5a8fa7c1-2c25-4534-baa9-58e9c2206c6e,TransferInEvent,Depósito recebido,"R$ 150,00",2018-02-22,150.00,NaN,NaN
513,513,5a8d0b8c-534b-4268-ae8b-cb8cf5a893c6,TransferInEvent,Depósito recebido,"R$ 350,00",2018-02-20,350.00,NaN,NaN
